In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

def request_url(urldata, endpage=5):            # url 생성 모듈
    user_key = "706363516d73696e37324744735257"
    url = "http://openapi.seoul.go.kr:8088/"
    url = url + user_key
    url = url + "/xml/CardSubwayStatsNew/1/" + str(endpage) + "/" + str(urldata)

    #print(url)
    
    subway_html = requests.get(url)
    
    if subway_html.status_code != 200:
        exit("데이터를 받지 못함")
    return subway_html

def swdataframe(schdata):                     # 요청 데이터를 이용해 DataFrame으로 제공
    subway_html=request_url(schdata)
    soup = bs(subway_html.text, "html.parser")
    #print(soup)

    endpage = soup.find('list_total_count').get_text()

    subway_html=request_url(schdata, endpage)
    soup = bs(subway_html.text, "html.parser")

    rows = soup.find_all("row")

    sw = []
    for row in rows:
        use_dt = row.find('use_dt').get_text()
        line_num = row.find('line_num').get_text()
        sub_sta_nm = row.find('sub_sta_nm').get_text()
        ride_pasgr = row.find('ride_pasgr_num').get_text()
        alight_pasgr = row.find('alight_pasgr_num').get_text()

        sw.append([use_dt, line_num, sub_sta_nm, ride_pasgr, alight_pasgr])

    swdf = pd.DataFrame(sw, columns=["사용일자", "호선명", "역명", "승차인원", "하차인원"])
    return swdf

# 현재 위치에서 실행시에만 아래 코드가 실행
# 즉 다른 파일에서 호출하면 실행 안됨
if __name__ == "__main__":
    schdata = input("조회일을 입력하세요.(ex:20220101):")
    df1 = swdataframe(schdata)
    df1.head()